<a href="https://colab.research.google.com/github/GuptaNavdeep1983/CS767/blob/main/Assignment7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import keras
import tensorflow as tf
import numpy as np
import math
from sklearn.model_selection import StratifiedKFold, train_test_split

In [2]:
from keras.datasets import cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()


170508288/170498071 [==============================] - 5s 0us/step


In [8]:
InitSeed = 767
tf.random.set_seed(InitSeed)
np.random.seed( InitSeed)


In [9]:
x_train = tf.map_fn(lambda frame: tf.image.per_image_standardization(frame), x_train, dtype=tf.float32).numpy()

In [10]:
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.1, random_state=42, stratify=y_train)

In [11]:
def step_decay1(epoch):
   initial_lrate = 0.001
   drop = 0.0001
   epochs_drop = 10.0
   lrate = initial_lrate * math.pow(drop,  
           math.floor((1+epoch)/epochs_drop))
   return lrate
lrate1 = tf.keras.callbacks.LearningRateScheduler(step_decay1)

In [12]:
def step_decay2(epoch, lr):
   if epoch < 10:
     return lr
   else:
     return lr * tf.math.exp(-0.1)

lrate2 = tf.keras.callbacks.LearningRateScheduler(step_decay2)

In [21]:
checkpoint_cb = keras.callbacks.ModelCheckpoint ("my_cifar10.model.h5" , save_best_only=True)
earlystopping_cb = keras.callbacks.EarlyStopping (patience=10, restore_best_weights=True)

epochs = 50
learning_rate = 0.001
decay_rate = learning_rate / epochs
momentum = 0.8

input1 = keras.layers.Flatten(input_shape=[32,32,3])
hidden1 = keras.layers.Dense(2048, activation='relu', name="Hidden1")
hidden2 = keras.layers.Dense(1024, activation='relu', name="Hidden2")
hidden3 = keras.layers.Dense(512, activation='relu', name="Hidden3")
output = keras.layers.Dense(10, activation='softmax', name="Output")

model1 = keras.models.Sequential([input1, hidden1, hidden2, hidden3, output])

sgd = keras.optimizers.SGD(learning_rate=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)

model1.compile(loss='sparse_categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])
print(model1.optimizer.learning_rate)
model1.fit(x_train[train], y_train[train],epochs=100, batch_size=128, validation_data=(x_train[test], y_train[test]), verbose=2, callbacks=[earlystopping_cb, checkpoint_cb, lrate2])

<tf.Variable 'learning_rate:0' shape=() dtype=float32, numpy=0.001>
Epoch 1/100
1407/1407 - 21s - loss: 1.6922 - accuracy: 0.4048 - val_loss: 1.5308 - val_accuracy: 0.4542
Epoch 2/100
1407/1407 - 20s - loss: 1.4781 - accuracy: 0.4763 - val_loss: 1.5114 - val_accuracy: 0.4640
Epoch 3/100
1407/1407 - 21s - loss: 1.3774 - accuracy: 0.5110 - val_loss: 1.4544 - val_accuracy: 0.4798
Epoch 4/100
1407/1407 - 19s - loss: 1.2848 - accuracy: 0.5464 - val_loss: 1.4751 - val_accuracy: 0.4798
Epoch 5/100
1407/1407 - 19s - loss: 1.2053 - accuracy: 0.5732 - val_loss: 1.4516 - val_accuracy: 0.5020
Epoch 6/100
1407/1407 - 20s - loss: 1.1218 - accuracy: 0.6026 - val_loss: 1.5102 - val_accuracy: 0.4954
Epoch 7/100
1407/1407 - 19s - loss: 1.0430 - accuracy: 0.6309 - val_loss: 1.5697 - val_accuracy: 0.5024
Epoch 8/100
1407/1407 - 19s - loss: 0.9707 - accuracy: 0.6567 - val_loss: 1.5846 - val_accuracy: 0.5018
Epoch 9/100
1407/1407 - 19s - loss: 0.8959 - accuracy: 0.6824 - val_loss: 1.6041 - val_accuracy: 0.5